In [168]:
import os

In [169]:
%pwd

'c:\\'

In [170]:
os.chdir("../")

In [171]:
%pwd

'c:\\'

In [172]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [173]:
from GGDDClassifier.constants import *
from GGDDClassifier.utils.common import read_yaml, create_directories

In [174]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      


In [175]:
import os
import urllib.request as request
import zipfile
from GGDDClassifier import logger
from GGDDClassifier.utils.common import get_size

In [176]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [182]:
import os
import requests
import zipfile

class DataIngestion:
    def __init__(self, config):
        self.config = config

    def download_file(self):
        source_url = self.config['source_URL']
        local_data_file = self.config['local_data_file']
        response = requests.get(source_url)
        with open(local_data_file, 'wb') as file:
            file.write(response.content)

    def extract_zip_file(self):
        local_data_file = self.config['local_data_file']
        unzip_dir = self.config['unzip_dir']
        with zipfile.ZipFile(local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)

# Your existing code with the DataIngestion class defined

current_directory = os.getcwd()
artifacts_directory = os.path.join(current_directory, 'artifacts')
data_ingestion_config = {
    'artifacts_root': artifacts_directory,
    'data_ingestion': {
        'root_dir': current_directory,
        'source_URL': 'https://github.com/spy-25/datasets/raw/main/chicken_fecal_images.zip',
        'local_data_file': os.path.join(artifacts_directory, 'data.zip'),
        'unzip_dir': artifacts_directory
    }
}

try:
    os.makedirs(artifacts_directory, exist_ok=True)
    data_ingestion_config_inner = data_ingestion_config['data_ingestion']
    data_ingestion = DataIngestion(config=data_ingestion_config_inner)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    print("An error occurred:", e)

In [ ]:
import os
import platform
import requests
import zipfile
import subprocess

class DataIngestion:
    def __init__(self, config):
        self.config = config

    def download_file(self):
        source_url = self.config['source_URL']
        local_data_file = self.config['local_data_file']
        response = requests.get(source_url)
        with open(local_data_file, 'wb') as file:
            file.write(response.content)

    def extract_zip_file(self):
        local_data_file = self.config['local_data_file']
        unzip_dir = self.config['unzip_dir']
        with zipfile.ZipFile(local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)

def open_folder(directory):
    system = platform.system()
    if system == 'Windows':
        os.startfile(directory)
    elif system == 'Darwin':  # macOS
        subprocess.Popen(['open', directory])
    elif system == 'Linux':
        subprocess.Popen(['xdg-open', directory])

# Your existing code with the DataIngestion class defined

current_directory = os.getcwd()
artifacts_directory = os.path.join(current_directory, 'artifacts')
data_ingestion_config = {
    'artifacts_root': artifacts_directory,
    'data_ingestion': {
        'root_dir': current_directory,
        'source_URL': 'https://github.com/spy-25/datasets/raw/main/chicken_fecal_images.zip',
        'local_data_file': os.path.join(artifacts_directory, 'data.zip'),
        'unzip_dir': artifacts_directory
    }
}

try:
    os.makedirs(artifacts_directory, exist_ok=True)
    data_ingestion_config_inner = data_ingestion_config['data_ingestion']
    data_ingestion = DataIngestion(config=data_ingestion_config_inner)
    data_ingestion.download_file()
    print("File downloaded successfully.")
    data_ingestion.extract_zip_file()
    print("Extraction completed successfully. Opening folder...")
    open_folder(artifacts_directory)
except Exception as e:
    print("An error occurred:", e)
